# Аналог первичного ключа

In [1]:
import pandas as pd

In [2]:
logs = pd.read_csv('logs.csv', sep='\t')
logs.head()

,CounterID,EventDate,EventID,Country,visits
0,333,2020-11-18,99905,Russia,244
1,555,2020-09-14,99892,Other,4138
2,111,2020-01-08,99872,Russia,7830
3,333,2020-04-17,99869,Other,9675
4,555,2020-09-29,99837,Other,1313


In [10]:
logs.CounterID.nunique()

5

In [11]:
logs.EventDate.nunique()

366

In [12]:
5 * 366

1830

### Хотим посчитать сумму визитов для счетчика 333 за 2020-07-01
Попробуем разные способы простого перебора всей таблицы

In [3]:
# вариант 1
logs[(logs.CounterID == 333) & (logs.EventDate == '2020-07-01')].visits.sum()

13690

In [4]:
# вариант 2
logs.loc[(logs.CounterID == 333) & (logs.EventDate == '2020-07-01'), 'visits'].sum()

13690

In [5]:
visits_sum = 0

with open('logs.csv') as f:
    f.readline()  # пропускаем заголовок
    
    for line in f:
        counter_id, event_date, event_id, country, visits = line.strip().split('\t')
        
        if counter_id == '333' and event_date == '2020-07-01':
            visits_sum += int(visits)
            
visits_sum

13690

Оценим скорость работы:

In [6]:
%%time

for _ in range(10**4):
    logs[(logs.CounterID == 333) & (logs.EventDate == '2020-07-01')].visits.sum()

CPU times: user 16.2 s, sys: 176 ms, total: 16.4 s
Wall time: 18.4 s


In [7]:
%%time

for _ in range(10**4):
    logs.loc[(logs.CounterID == 333) & (logs.EventDate == '2020-07-01'), 'visits'].sum()

CPU times: user 13.7 s, sys: 155 ms, total: 13.9 s
Wall time: 15.5 s


In [8]:
%%time

for _ in range(10**4):
    visits_sum = 0

    with open('logs.csv') as f:
        f.readline()  # пропускаем заголовок

        for line in f:
            counter_id, event_date, event_id, country, visits = line.strip().split('\t')

            if counter_id == '333' and event_date == '2020-07-01':
                visits_sum += int(visits)

CPU times: user 27.5 s, sys: 1.15 s, total: 28.7 s
Wall time: 34.8 s


### Используем первичный ключ
Точнее, аналог в виде словаря

In [13]:
logs.sort_values(['CounterID', 'EventDate']).to_csv('logs_sorted_by_pk.csv', index=False)

In [15]:
logs.sort_values(['CounterID', 'EventDate']).head(20)

,CounterID,EventDate,EventID,Country,visits
18,111,2020-01-01,99641,Russia,8980
1180,111,2020-01-01,70725,Other,6131
1130,111,2020-01-02,72177,Other,6862
2599,111,2020-01-02,36003,Russia,1793
760,111,2020-01-03,81308,Other,3205
2794,111,2020-01-03,30923,Russia,900
1541,111,2020-01-04,60696,Other,2139
1833,111,2020-01-04,53500,Russia,5905
1030,111,2020-01-05,74816,Other,3721
1256,111,2020-01-05,68888,Russia,6948


In [16]:
logs_dict = {}
prev_counter = None
prev_date = None

with open('logs_sorted_by_pk.csv') as f:
    f.readline()
    
    for i, line in enumerate(f):
        counter_id, event_date, event_id, country, visits = line.strip().split(',')
        
        if prev_counter:
            if counter_id == prev_counter and event_date == prev_date:
                current_pk_data.append((event_id, country, int(visits)))
            else:
                logs_dict[(prev_counter, prev_date)] = current_pk_data
                current_pk_data = [(event_id, country, int(visits))]
                
        else:
            current_pk_data = [(event_id, country, int(visits))]

        prev_counter = counter_id
        prev_date = event_date

logs_dict[(prev_counter, prev_date)] = current_pk_data

In [19]:
for i in logs_dict[('333', '2020-07-01')]:
    print(i)

('99346', 'Russia', 4641)
('77639', 'Other', 9049)


In [20]:
sum([x[2] for x in logs_dict[('333', '2020-07-01')]])

13690

In [21]:
%%time

for _ in range(10**4):
    sum([x[2] for x in logs_dict[('333', '2020-07-01')]])

CPU times: user 6.38 ms, sys: 125 µs, total: 6.51 ms
Wall time: 8.44 ms


In [ ]:
logs_dict

# Создание таблицы в Кликхаусе из файла логов
Копируем файл logs_no_header.csv в контейнер container_id (можно узнать через команду docker ps)
```
docker cp logs_no_header.csv container_id:/
```

Заходим в контейнер
```
docker exec -it container_id bash
```

Заходим в клиент Кликхауса
```
clickhouse-client
```

Используем любую базу данных (либо создаем ее create database datasets)
```
use datasets
```

Создаем таблицу logs
```
CREATE TABLE logs
(
    `CounterID` UInt32,
    `EventDate` Date,
    `EventID` UInt32,
    `Country` String,
    `Visits` UInt32
)
ENGINE = MergeTree
PARTITION BY toYYYYMM(EventDate)
ORDER BY (CounterID, EventDate)
```

Заливаем данные из файла logs_no_header.csv в таблицу logs:
```
cat logs_no_header.csv | clickhouse-client --database datasets --query "INSERT INTO logs FORMAT TSV"
```

Проверяем первые 5 строк:
```
clickhouse-client --database datasets --query 'select * from logs limit 5'
```

На выходе получим уже отсортированные по CounterID, EventDate строки:
```
111	2020-01-01	99641	Russia	8980
111	2020-01-01	70725	Other	6131
111	2020-01-02	72177	Other	6862
111	2020-01-02	36003	Russia	1793
111	2020-01-03	81308	Other	3205
```